In [1]:
from datasets import DatasetDict, Dataset
import json

/Users/yuxuan/.pyenv/versions/3.11.5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
medqa_system = "You are a medical doctor taking the US Medical Licensing Examination. You need to demonstrate your understanding of basic and clinical science, medical knowledge, and mechanisms underlying health, disease, patient care, and modes of therapy. Show your ability to apply the knowledge essential for medical practice. For the following multiple-choice question, select one correct answer from A to E. Base your answer on the current and standard practices referenced in medical guidelines."

# {"conversations": [{"from": "...", "value": "..."}]}
# chat = [
#     {"from": "user", "value": user},
#     {"from": "assistant", "value": assistant},
# ]
# row["conversations"] = chat


def format_medqa_chat(input_list):
    constructed = {}
    chats = []
    for i in range(len(input_list)):
        question = input_list[i]["question"]
        options = input_list[i]["options"]
        answer = input_list[i]["answer"]
        answer_key = [k for k, v in options.items() if v == answer][0]
        formatted_options = "\n".join([f"({k}) {v}" for k, v in options.items()])
        chat = [
            {
                "from": "system",
                "value": medqa_system,
            },
            {
                "from": "user",
                "value": f"Question: {question}\nOptions:\n{formatted_options}",
            },
            {"from": "assistant", "value": f"({answer_key}) {answer}"},
        ]
        chats.append(chat)
    constructed["conversations"] = chats
    return constructed


data_train = []


with open("train.jsonl", "r") as f:
    for line in f:
        data_train.append(json.loads(line))

# data_train[5]

dataset = Dataset.from_dict(format_medqa_chat(data_train))


dataset.push_to_hub("Medilora/MedMCQA-ShareGPT", private=True)

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:21<00:00, 21.16s/it]


In [2]:
from datasets import load_dataset

dataset = load_dataset("Medilora/MedMCQA-ShareGPT")

Generating test split: 100%|██████████| 2/2 [00:00<00:00, 1770.12 examples/s]


In [6]:
dataset["train"][1]

{'conversations': {'from': 'assistant',
  'value': '(D) Partial molar pregnancy'}}

In [ ]:
data_train = []

with open("US_qbank.jsonl", "r") as f:
    for line in f:
        data_train.append(json.loads(line))

unique_keys = set()

for entry in data_train:
    for key in entry.keys():
        unique_keys.add(key)

print(f"There are {len(unique_keys)} unique keys in the JSON data.")
print(f"The unique keys are: {unique_keys}")

data_train[5]